In [ ]:
{
  "cells": [
    {
      "cell_type": "code",
      "source": [
        "import numpy as np\n",
        "import copy as cp\n",
        "from math import floor, ceil\n",
        "import io\n",
        "import os\n",
        "import copy"
      ],
      "outputs": [],
      "execution_count": 1,
      "metadata": {
        "collapsed": true,
        "jupyter": {
          "source_hidden": false,
          "outputs_hidden": false
        },
        "nteract": {
          "transient": {
            "deleting": false
          }
        },
        "execution": {
          "iopub.status.busy": "2022-04-02T18:59:12.928Z",
          "iopub.execute_input": "2022-04-02T18:59:12.932Z",
          "iopub.status.idle": "2022-04-02T18:59:13.022Z",
          "shell.execute_reply": "2022-04-02T18:59:12.998Z"
        }
      }
    },
    {
      "cell_type": "code",
      "source": [
        "anchor = \"\"\"<a name=\"%s\"></a><br/><br/><br/>\\n\"\"\"\n",
        "\n",
        "player_map = {\n",
        "  'X': '✖️',\n",
        "  'O': '⭕',\n",
        "  '_': '⬜'\n",
        "}\n",
        "\n",
        "play_again = \"[Care to play again?](#root)\"\n",
        "draw_message = \"Guess we'll call it a draw\"\n",
        "win_message = \"%s wins!!!\"\n",
        "\n",
        "header = \"\"\"\n",
        "<a name=\"top\"></a>\n",
        "I challenge you to a game of Tic-Tac-Toe. \n",
        "Give this page a few seconds to load all the emojis and then click the topmost box to start playing.\n",
        "\n",
        "Xs go first, click one of the nine boxes to make your move.\n",
        "\n",
        "_If for some reason the buttons are not clickable, try reloading the page. Sometimes GitHub can't handle all of the emojis._\n",
        "\n",
        "[Take me to the bottom](#bottom)\n",
        "\n",
        "---\n",
        "\"\"\"\n",
        "\n",
        "footer = \"\"\"\n",
        "\n",
        "---\n",
        "\n",
        "<a name=\"bottom\"></a>\n",
        "[Take me to the top](#top)\n",
        "\"\"\""
      ],
      "outputs": [],
      "execution_count": 2,
      "metadata": {
        "collapsed": true,
        "jupyter": {
          "source_hidden": false,
          "outputs_hidden": false
        },
        "nteract": {
          "transient": {
            "deleting": false
          }
        },
        "execution": {
          "iopub.status.busy": "2022-04-02T18:59:13.028Z",
          "iopub.execute_input": "2022-04-02T18:59:13.034Z",
          "iopub.status.idle": "2022-04-02T18:59:13.038Z",
          "shell.execute_reply": "2022-04-02T18:59:13.015Z"
        }
      }
    },
    {
      "cell_type": "code",
      "source": [
        "class State:\n",
        "    def __init__(self, moves):\n",
        "        self.to_move = 'X'\n",
        "        self.utility = 0\n",
        "        self.board = {}\n",
        "        self.moves = cp.copy(moves)\n",
        "        \n",
        "class TicTacToe:\n",
        "    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):\n",
        "        self.nrow = nrow\n",
        "        self.ncol = ncol\n",
        "        self.nwin = nwin\n",
        "        self.nexp = nexp\n",
        "        self.state = State([(r, c) for r in range(1, nrow+1) for c in range(1, ncol+1)])        \n",
        "\n",
        "    def result(self, move, state):\n",
        "        if move not in state.moves:\n",
        "            return state\n",
        "        new_state = cp.deepcopy(state)\n",
        "        new_state.utility = self.compute_utility(move, state)\n",
        "        new_state.board[move] = state.to_move\n",
        "        new_state.moves.remove(move)\n",
        "        new_state.to_move = ('O' if state.to_move == 'X' else 'X')\n",
        "        return new_state\n",
        "        \n",
        "    def compute_utility(self, move, state):\n",
        "        row, col = move\n",
        "        player = state.to_move\n",
        "        \n",
        "        board = cp.deepcopy(state.board)\n",
        "        board[move] = player\n",
        "\n",
        "        in_a_row = 0\n",
        "        for c in range(1,self.ncol+1):\n",
        "            in_a_row += board.get((row,c))==player\n",
        "        in_a_col = 0\n",
        "        for r in range(1,self.nrow+1):\n",
        "            in_a_col += board.get((r,col))==player\n",
        "        in_a_diag1 = 0\n",
        "        for r in range(row,0,-1):\n",
        "            in_a_diag1 += board.get((r,col-(row-r)))==player\n",
        "        for r in range(row+1,self.nrow+1):\n",
        "            in_a_diag1 += board.get((r,col-(row-r)))==player\n",
        "        in_a_diag2 = 0\n",
        "        for r in range(row,0,-1):\n",
        "            in_a_diag2 += board.get((r,col+(row-r)))==player\n",
        "        for r in range(row+1,self.nrow+1):\n",
        "            in_a_diag2 += board.get((r,col+(row-r)))==player\n",
        "            \n",
        "        moves_cost = (len(state.moves) + 1) * 0.01\n",
        "        \n",
        "        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:\n",
        "            return (1+moves_cost)*(1 if player=='X' else -1)\n",
        "        else:\n",
        "            return 0\n",
        "\n",
        "    def game_over(self, state):\n",
        "        return state.utility!=0 or len(state.moves)==0\n",
        "\n",
        "    def utility(self, state, player):\n",
        "        return state.utility if player=='X' else -state.utility\n",
        "\n",
        "    def move(self, move):\n",
        "        self.state = self.result(move, self.state)\n",
        "\n",
        "def alphabeta_search(game):\n",
        "    player = game.state.to_move\n",
        "    \n",
        "    def max_value(state, alpha=-float('inf'), beta=float('inf')):\n",
        "        if game.game_over(state):\n",
        "            return game.utility(state, player), None\n",
        "        \n",
        "        bestVal = -float('inf')\n",
        "        bestMove = None\n",
        "        \n",
        "        for move in state.moves:\n",
        "            value, _ = min_value(game.result(move, state), alpha, beta)\n",
        "            if value > bestVal:\n",
        "                bestMove = move\n",
        "            bestVal = max(bestVal, value)\n",
        "            alpha = max(alpha, bestVal)\n",
        "            if beta <= alpha:\n",
        "                break\n",
        "        return bestVal, bestMove\n",
        "\n",
        "    def min_value(state, alpha=-float('inf'), beta=float('inf')):\n",
        "        if game.game_over(state):\n",
        "            return game.utility(state, player), None\n",
        "        \n",
        "        bestVal = float('inf')\n",
        "        bestMove = None\n",
        "        \n",
        "        for move in state.moves:\n",
        "            value, _ = max_value(game.result(move, state), alpha, beta)\n",
        "            if value < bestVal:\n",
        "                bestMove = move\n",
        "            bestVal = min(bestVal, value)\n",
        "            beta = min(beta, bestVal)\n",
        "            if beta <= alpha:\n",
        "                break\n",
        "        return bestVal, bestMove\n",
        "    \n",
        "    _, move = max_value(game.state)\n",
        "    return move"
      ],
      "outputs": [],
      "execution_count": 10,
      "metadata": {
        "collapsed": true,
        "jupyter": {
          "source_hidden": false,
          "outputs_hidden": false
        },
        "nteract": {
          "transient": {
            "deleting": false
          }
        },
        "execution": {
          "iopub.status.busy": "2022-04-02T19:01:24.420Z",
          "iopub.execute_input": "2022-04-02T19:01:24.424Z",
          "iopub.status.idle": "2022-04-02T19:01:24.431Z",
          "shell.execute_reply": "2022-04-02T19:01:24.435Z"
        }
      }
    },
    {
      "cell_type": "code",
      "source": [
        "class GameState:\n",
        "  def __init__(self, moves = []):\n",
        "    self.children = []\n",
        "    self.moves = moves\n",
        "    self.winner = None\n",
        "    self.terminal = False\n",
        "    self.id = self.make_id()\n",
        "  \n",
        "  def make_id(self):\n",
        "    if len(self.moves) != 0:\n",
        "      return \"-\".join([\"%d%d\" % move for move in self.moves])\n",
        "    else:\n",
        "      return \"root\"\n",
        "    \n",
        "  def add_child(self, move):\n",
        "    child = GameState([*self.moves, move])\n",
        "    self.children.append(child)\n",
        "    return child\n",
        "  \n",
        "  def get_child_state(self, move):\n",
        "    return next((child for child in self.children if child.moves == [*self.moves, move]), None)\n",
        "  \n",
        "  def make_state_map(self, moves):\n",
        "    state = {}\n",
        "    player = 'X'\n",
        "    for move in moves: \n",
        "      state[move] = player\n",
        "      player = 'O' if player == 'X' else 'X'\n",
        "    return state\n",
        "\n",
        "def generate_games(game, parent_state):\n",
        "  if len(game.state.moves) == 0:\n",
        "    parent_state.terminal = True\n",
        "    return\n",
        "  \n",
        "  if game.game_over(game.state):\n",
        "    parent_state.winner = 'X' if game.state.to_move == 'O' else 'O'\n",
        "    parent_state.terminal = True\n",
        "    return\n",
        "  \n",
        "  if game.state.to_move == 'X':\n",
        "    for move in game.state.moves:\n",
        "      new_game = copy.deepcopy(game)\n",
        "      new_game.state = new_game.result(move, new_game.state)\n",
        "      generate_games(new_game, parent_state.add_child(move))\n",
        "  else:\n",
        "    new_game = copy.deepcopy(game)\n",
        "    move = alphabeta_search(game)\n",
        "    new_game.state = new_game.result(move, new_game.state)\n",
        "    generate_games(new_game, parent_state.add_child(move))\n",
        "\n",
        "   \n",
        "t = TicTacToe(nrow=3, ncol=3, nwin=3)\n",
        "root = GameState()\n",
        "generate_games(t, root)"
      ],
      "outputs": [],
      "execution_count": 11,
      "metadata": {
        "collapsed": true,
        "jupyter": {
          "source_hidden": false,
          "outputs_hidden": false
        },
        "nteract": {
          "transient": {
            "deleting": false
          }
        },
        "execution": {
          "iopub.status.busy": "2022-04-02T19:01:27.032Z",
          "iopub.execute_input": "2022-04-02T19:01:27.110Z",
          "iopub.status.idle": "2022-04-02T19:01:31.107Z",
          "shell.execute_reply": "2022-04-02T19:01:31.139Z"
        }
      }
    },
    {
      "cell_type": "code",
      "source": [
        "short_ids = {}\n",
        "id_index = 0\n",
        "def get_short_id(key):\n",
        "  \n",
        "  if key == \"root\":\n",
        "    return key\n",
        "  \n",
        "  global short_ids\n",
        "  global id_index\n",
        "  if key in short_ids:\n",
        "    return short_ids[key]\n",
        "  else:\n",
        "    id_index += 1\n",
        "    short_ids[key] = id_index\n",
        "    return id_index\n",
        "\n",
        "def node_str(node):\n",
        "  state = node.make_state_map(node.moves)\n",
        "  result = anchor % get_short_id(node.id)\n",
        "  \n",
        "  use_links = len(node.children) != 0 and node.winner is None\n",
        "  \n",
        "  for row in range(1, 3+1):\n",
        "    line = \"\"\n",
        "    for col in range(1, 3+1):\n",
        "      player_for_position = state.get((row, col), '_')\n",
        "      \n",
        "      display_player = player_map[player_for_position]\n",
        "      \n",
        "      if use_links and player_for_position == '_':\n",
        "        o_move = node.get_child_state((row, col))\n",
        "        if len(o_move.children) != 0:\n",
        "          next_id = get_short_id(o_move.children[0].id)\n",
        "        else:\n",
        "          next_id = get_short_id(o_move.id)\n",
        "        line += \"[%s](#%s) \" % (display_player, next_id)\n",
        "      else:\n",
        "        line += \"%s \" % display_player\n",
        "    result += \"%s<br/>\" % line.rstrip()\n",
        "  \n",
        "  if node.winner is not None:\n",
        "    result += win_message % node.winner\n",
        "  elif len(node.moves) == 9:\n",
        "    result += draw_message\n",
        "  \n",
        "  if node.terminal:\n",
        "    result += \"<br/>%s\" % play_again\n",
        "    \n",
        "  result += \"<br/><br/><br/>\"\n",
        "  return result\n",
        "\n",
        "node_count = 0\n",
        "def bfs_display_v2(start):\n",
        "  global node_count\n",
        "  output = \"\"\n",
        "  queue = [start]\n",
        "  while len(queue) != 0:\n",
        "    visit = queue.pop(0)\n",
        "    queue.extend(visit.children)\n",
        "    \n",
        "    if len(visit.moves) % 2 == 0 or visit.terminal:\n",
        "      node_count+=1\n",
        "      output += \"%s\\n\" % node_str(visit)\n",
        "  return output\n",
        "    \n",
        "dfs = bfs_display_v2(root)\n",
        "\n",
        "with io.open('README.md', 'w', encoding='utf-8') as f:\n",
        "  f.write(header)\n",
        "  f.write(dfs)\n",
        "  f.write(footer)\n",
        "\n",
        "file_size_kb = os.path.getsize('README.md') / 1024\n",
        "print(\"Done\", \"%skb\" % round(ceil(file_size_kb)), node_count, \"nodes\")"
      ],
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Done 138kb 952 nodes\n"
          ]
        }
      ],
      "execution_count": 12,
      "metadata": {
        "collapsed": false,
        "jupyter": {
          "source_hidden": false,
          "outputs_hidden": false
        },
        "nteract": {
          "transient": {
            "deleting": false
          }
        },
        "execution": {
          "iopub.status.busy": "2022-04-02T19:01:31.114Z",
          "iopub.execute_input": "2022-04-02T19:01:31.118Z",
          "iopub.status.idle": "2022-04-02T19:01:31.130Z",
          "shell.execute_reply": "2022-04-02T19:01:31.143Z"
        }
      }
    }
  ],
  "metadata": {
    "kernel_info": {
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.9.2",
      "mimetype": "text/x-python",
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "pygments_lexer": "ipython3",
      "nbconvert_exporter": "python",
      "file_extension": ".py"
    },
    "kernelspec": {
      "argv": [
        "C:\\Python37\\python.exe",
        "-m",
        "ipykernel_launcher",
        "-f",
        "{connection_file}"
      ],
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "nteract": {
      "version": "0.28.0"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}